# Interoperability

```{admonition} Summary

- Interoperabilty between languages allows analysts to take advantage of the strengths of different ecosystems
- On-disk interoperability uses standard file formats to transfer data and is typically more reliable
- In-memory interoperabilty transfers data directly between parallel sessions and is convenient for interactive analysis
- While interoperability is currently possible developers continue to improve the experience
```

## Motivation

As we have discussed in the {ref}`analysis frameworks and tools chapter <introduction:analysis-frameworks>` there are three main ecosystems for single-cell analysis, the [Bioconductor](https://bioconductor.org/) and [Seurat](https://satijalab.org/seurat/index.html) ecosystems in R and the Python-based [scverse](https://scverse.org/) ecosystem. A common question from new analysts is which ecosystem they should focus on learning and using? While it makes sense to focus on one to start with, and a successful standard analysis can be performed in any ecosystem, we promote the idea that competent analysts should be familiar with all three ecosystems and comfortable moving between them. This approach allows analysts to use the best-performing tools and methods regardless of how they were implemented. When analysts are not comfortable moving between ecosystems they often tend to use packages that are easy to access, even when they have been shown to have shortcomings compared to packages in another ecosystem. The ability of analysts to move between ecosystems allows developers to take advantage of the different strengths of programming languages. For example, R has strong inbuilt support for complex statistical modelling while the majority of deep learning libraries are focused on Python. By supporting common on-disk data formats and in-memory data structures developers can be confident that analysts can access their package and can use the platform that is most appropriate for their method. Another motivation for being comfortable with multiple ecosystems is the accessibility and availability of data, results and documentation. Often data or results are only made available in one format and analysts will need to be familiar with that format in order to access it. A basic understanding of other ecosystems is also necessary to understand package documentation and tutorials when deciding which methods to use.

While we encourage analysts to be comfortable with all the major ecosystems, moving between them is only possible when they are interoperable. Thankfully, lots of work has been done in this area and it is now relatively simple in most cases using standard packages. In this chapter, we discuss the various ways data can be moved between ecosystems via disk or in-memory, the differences between them and their advantages. We focus on single-modality data and moving between R and Python as these are the most common cases but we also touch on multimodal data and other languages.

## Nomenclature

Because talking about different languages can get confusing we try to use the following conventions:

- **{package}** - An R package
- `package::function()` - A function in an R package
- **package** - A Python package
- `package.function()` - A function in a Python package
- **Emphasised** - Some other important concept
- `code` - Other parts of code including objects, variables etc. This is also used for files or directories.

In [1]:
import tempfile
from pathlib import Path

import anndata
import anndata2ri
import mudata
import numpy
import rpy2.robjects
import scanpy
from scipy.sparse import csr_matrix

anndata2ri.activate()
%load_ext rpy2.ipython

## Disk-based interoperability

The first approach to moving between languages is via disk-based interoperability. This involves writing a file to disk in one language and then reading that file into a second language. In many cases, this approach is simpler, more reliable and scalable than in-memory interoperability (which we discuss below) but it comes at the cost of greater storage requirements and reduced interactivity. Disk-based interoperability tends to work particularly well when there are established processes for each stage of analysis and you want to pass objects from one to the next (especially as part of a pipeline developed using a workflow manager such as [Nextflow](https://www.nextflow.io/index.html) or [snakemake](https://snakemake.readthedocs.io/en/stable/)). However, disk-based interoperability is less convenient for interactive steps such as data exploration or experimenting with methods as you need to write a new file whenever you want to move between languages.

### Simple formats

Before discussing file formats specifically developed for single-cell data we want to briefly mention that common simple text file formats (such as CSV, TSV, JSON etc.) can often be the answer to transferring data between languages. They work well in cases where some analysis has been performed and what you want to transfer is a subset of the information about an experiment. For example, you may want to transfer only the cell metadata but do not require the feature metadata, expression matrices etc. The advantage of using simple text formats is that they are well supported by almost any language and do not require single-cell specific packages. However, they can quickly become impractical as what you want to transfer becomes more complex.

### HDF5-based formats

The most common disk formats for single-cell data are based on [Hierarchical Data Format version 5](https://www.hdfgroup.org/solutions/hdf5/) or HDF5. This is an open-source file format designed for storing large, complex and heterogeneous data. It has a file directory type structure (similar to how files and folders are organised on your computer) which allows many different kinds of data to be stored in a single file with an arbitrarily complex hierarchy. While this format is very flexible, to properly interact with it you need to know where and how the different information is stored. For this reason, standard specifications for storing single-cell data in HDF5 files have been developed.

#### H5AD

The H5AD format is the HDF5 disk representation of the `AnnData` object used by scverse packages and is commonly used to share single-cell datasets. As it is part of the scverse ecosystem, reading and writing these files from Python is well-supported and is part of the core functionality of the [**anndata** package](https://anndata.readthedocs.io/en/latest/index.html) (read more about the format [here](https://anndata.readthedocs.io/en/latest/fileformat-prose.html)).

To demonstrate interoperability we will use a small, randomly generated dataset that has gone through some of the steps of a standard analysis workflow to populate the various slots.

In [3]:
# Create a randomly generated AnnData object to use as an example
counts = csr_matrix(
    numpy.random.default_generator().poisson(1, size=(100, 2000)), dtype=numpy.float32
)
adata = anndata.AnnData(counts)
adata.obs_names = [f"Cell_{i:d}" for i in range(adata.n_obs)]
adata.var_names = [f"Gene_{i:d}" for i in range(adata.n_vars)]
# Do some standard processing to populate the object
scanpy.pp.calculate_qc_metrics(adata, inplace=True)
adata.layers["counts"] = adata.X.copy()
scanpy.pp.normalize_total(adata, inplace=True)
scanpy.pp.log1p(adata)
scanpy.pp.highly_variable_genes(adata, inplace=True)
scanpy.tl.pca(adata)
scanpy.pp.neighbors(adata)
scanpy.tl.umap(adata)
adata

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg', 'pca', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'distances', 'connectivities'

We will write this mock object to disk as a H5AD file to demonstrate how those files can be read from R.

In [4]:
temp_dir = tempfile.TemporaryDirectory()
h5ad_file = Path(temp_dir.name) / "example.h5ad"

adata.write_h5ad(h5ad_file)

Several packages exist for reading and writing H5AD files from R. While they result in a file on disk these packages usually rely on wrapping the Python **anndata** package to handle the actual reading and writing of files with an in-memory conversion step to convert between R and Python.

##### Reading/writing H5AD with Bioconductor

The [Bioconductor **{zellkonverter}** package](https://bioconductor.org/packages/zellkonverter/) helps make this easier by using the [**{basilisk}** package](https://bioconductor.org/packages/basilisk/) to manage creating an appropriate Python environment. If that all sounds a bit technical, the end result is that Bioconductor users can read and write H5AD files using commands like below without requiring any knowledge of Python.

Unfortunately, because of the way this book is made, we are unable to run the code directly here. Instead, we will show the code and what the output looks like when run in an R session:

```r
sce <- zellkonverter::readH5AD(h5ad_file, verbose = TRUE)
```

```
ℹ Using the Python reader
ℹ Using anndata version 0.8.0
✔ Read /.../luke.zappia/Downloads/example.h5ad [113ms]
✔ uns$hvg$flavor converted [17ms]
✔ uns$hvg converted [50ms]
✔ uns$log1p converted [25ms]
✔ uns$neighbors converted [18ms]
✔ uns$pca$params$use_highly_variable converted [16ms]
✔ uns$pca$params$zero_center converted [16ms]
✔ uns$pca$params converted [80ms]
✔ uns$pca$variance converted [17ms]
✔ uns$pca$variance_ratio converted [16ms]
✔ uns$pca converted [184ms]
✔ uns$umap$params$a converted [16ms]
✔ uns$umap$params$b converted [16ms]
✔ uns$umap$params converted [80ms]
✔ uns$umap converted [112ms]
✔ uns converted [490ms]
✔ Converting uns to metadata ... done
✔ X matrix converted to assay [29ms]
✔ layers$counts converted [27ms]
✔ Converting layers to assays ... done
✔ var converted to rowData [25ms]
✔ obs converted to colData [24ms]
✔ varm$PCs converted [18ms]
✔ varm converted [47ms]
✔ Converting varm to rowData$varm ... done
✔ obsm$X_pca converted [15ms]
✔ obsm$X_umap converted [16ms]
✔ obsm converted [80ms]
✔ Converting obsm to reducedDims ... done
ℹ varp is empty and was skipped
✔ obsp$connectivities converted [22ms]
✔ obsp$distances converted [23ms]
✔ obsp converted [92ms]
✔ Converting obsp to colPairs ... done
✔ SingleCellExperiment constructed [164ms]
ℹ Skipping conversion of raw
✔ Converting AnnData to SingleCellExperiment ... done
```

Because we have turned on the verbose output you can see how **{zellkonverter}** reads the file using Python and converts each part of the `AnnData` object to a Bioconductor `SingleCellExperiment` object. We can see what the result looks like:

```r
sce
```

```
class: SingleCellExperiment
dim: 2000 100
metadata(5): hvg log1p neighbors pca umap
assays(2): X counts
rownames(2000): Gene_0 Gene_1 ... Gene_1998 Gene_1999
rowData names(11): n_cells_by_counts mean_counts ... dispersions_norm
  varm
colnames(100): Cell_0 Cell_1 ... Cell_98 Cell_99
colData names(8): n_genes_by_counts log1p_n_genes_by_counts ...
  pct_counts_in_top_200_genes pct_counts_in_top_500_genes
reducedDimNames(2): X_pca X_umap
mainExpName: NULL
altExpNames(0):
```

This object can then be used as normal by any Bioconductor package. If we want to write a new H5AD file we can use the `writeH5AD()` function:

```r
zellkonverter_h5ad_file <- tempfile(fileext = ".h5ad")
zellkonverter::writeH5AD(sce, zellkonverter_h5ad_file, verbose = TRUE)
```

```
ℹ Using anndata version 0.8.0
ℹ Using the 'X' assay as the X matrix
✔ Selected X matrix [29ms]
✔ assays$X converted to X matrix [50ms]
✔ additional assays converted to layers [30ms]
✔ rowData$varm converted to varm [28ms]
✔ reducedDims converted to obsm [68ms]
✔ metadata converted to uns [24ms]
ℹ rowPairs is empty and was skipped
✔ Converting AnnData to SingleCellExperiment ... done
✔ Wrote '/.../.../rj/.../T/.../file102cfa97cc51.h5ad ' [133ms]
```

We can then read this file in Python:

```python
scanpy.read_h5ad(zellkonverter_h5ad_file)
```

```
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'X_name', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```

If this the first time that you run a **{zellkonverter}** function you will see that it first creates a special conda environment to use (which can take a while). Once that environment exists it will be re-used by following function calls. **{zellkonverter}** has additional options such as allowing you to selectively read or write parts for an object. Please refer to the package documentation for more details. Similar functionality for writing a `SingleCellExperimentObject` to an H5AD file can be found in the [**{sceasy}** package](https://github.com/cellgeni/sceasy). While these packages are effective, wrapping Python requires some overhead which should be addressed by native R H5AD writers/readers in the future.

##### Reading/writing H5AD with **{Seurat}**

Converting between a `Seurat` object and an H5AD file is a two-step process [as suggested by this tutorial](https://mojaveazure.github.io/seurat-disk/articles/convert-anndata.html). Firstly H5AD file is converted to a H5Seurat file (a custom HDF5 format for `Seurat` objects) using the [**{SeuratDisk}** package](https://mojaveazure.github.io/seurat-disk/) and then this file is read as a `Seurat` object.

In [5]:
%%R -i h5ad_file

message("Converting H5AD to H5Seurat...")
SeuratDisk::Convert(h5ad_file, dest = "h5seurat", overwrite = TRUE)
message("Reading H5Seurat...")
h5seurat_file <- gsub(".h5ad", ".h5seurat", h5ad_file)
seurat <- SeuratDisk::LoadH5Seurat(h5seurat_file, assays = "RNA")
message("Read Seurat object:")
seurat


R[write to console]: Converting H5AD to H5Seurat...



R[write to console]: The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)




    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

R[write to console]: Warnung:
R[write to console]:  Unknown file type: h5ad

R[write to console]: Warnung:
R[write to console]:  'assay' not set, setting to 'RNA'

R[write to console]: Creating h5Seurat file for version 3.1.5.9900

R[write to console]: Adding X as data

R[write to console]: Adding X as counts

R[write to console]: Adding meta.features from var

R[write to console]: Adding X_pca as cell embeddings for pca

R[write to console]: Adding X_umap as cell embeddings for umap

R[write to console]: Adding PCs as feature loadings fpr pca

R[write to console]: Adding miscellaneous information for pca

R[write to console]: Adding standard deviations for pca

R[write to console]: Adding miscellaneous information for umap

R[write to console]: Adding hvg to miscellaneous data

R[write to console]: Adding log1p to miscellaneous data

R[write to console]: Adding

An object of class Seurat 
2000 features across 100 samples within 1 assay 
Active assay: RNA (2000 features, 0 variable features)
 2 dimensional reductions calculated: pca, umap


Note that because the structure of a `Seurat` object is quite different from `AnnData` and `SingleCellExperiment` objects the conversion process is more complex. See the [documentation of the conversion function](https://mojaveazure.github.io/seurat-disk/reference/Convert.html) for more details on how this is done.

The **{sceasy}** package also provides a function for reading H5AD files as `Seurat` or `SingleCellExperiment` objects in a single step. **{sceasy}** also wraps Python functions but unlike **{zellkonverter}** it doesn't use a special Python environment. This means you need to be responsible for setting up the environment, making sure that R can find it and that the correct packages are installed (again, this code is not run here).

```r
sceasy_seurat <- sceasy::convertFormat(h5ad_file, from="anndata", to="seurat")
sceasy_seurat
```
```
Warning: Feature names cannot have underscores ('_'), replacing with dashes ('-')
X -> counts
An object of class Seurat
2000 features across 100 samples within 1 assay
Active assay: RNA (2000 features, 0 variable features)
 2 dimensional reductions calculated: pca, umap
```

##### Reading/writing H5AD with **{anndata}**

The R [**{anndata}** package](https://anndata.dynverse.org/index.html) can also be used to read H5AD files. However, unlike the packages above it does not convert to a native R object. Instead it provides an R interface to the Python object. This is useful for accessing the data but few analysis packages will accept this as input so further in-memory conversion is usually required.

#### Loom

The [Loom file format](http://loompy.org/) is an older HDF5 specification for omics data. Unlike H5AD, it is not linked to a specific analysis ecosystem, although the structure is similar to `AnnData` and `SingleCellExperiment` objects. Packages implementing the Loom format exist for both [R](https://github.com/mojaveazure/loomR) and [Python](https://pypi.org/project/loompy/) as well as a [Bioconductor package](https://bioconductor.org/packages/LoomExperiment/) for writing Loom files. However, it is often more convenient to use the higher-level interfaces provided by the core ecosystem packages. Apart from sharing datasets another common place Loom files are encountered is when spliced/unspliced reads are quantified using [velocyto](http://velocyto.org/) for {ref}`RNA velocity analysis <trajectories:rna-velocity>`.

### RDS files

Another file format you may see used to share single-cell datasets is the RDS format. This is a binary format used to serialise arbitrary R objects (similar to Python Pickle files). As `SingleCellExperiment` and `Seurat` objects did not always have matching on-disk representations RDS files are sometimes used to share the results from R analyses. While this is ok within an analysis project we discourage its use for sharing data publicly or with collaborators due to the lack of interoperability with other ecosystems. Instead, we recommend using one of the HDF5 formats mentioned above that can be read from multiple languages.

### New on-disk formats

While HDF5-based formats are currently the standard for on-disk representations of single-cell data other newer technologies such as [Zarr](https://zarr.dev/) and [TileDB](https://tiledb.com/) have some advantages, particularly for very large datasets and other modalities. We expect specifications to be developed for these formats in the future which may be adopted by the community (**anndata** already provides support for Zarr files).

## In-memory interoperability

The second approach to interoperability is to work on in-memory representations of an object. This approach involves active sessions from two programming languages running at the same time and either accessing the same object from both or converting between them as needed. Usually, one language acts as the main environment and there is an interface to the other language. This can be very useful for interactive analysis as it allows an analyst to work in two languages simultaneously. It is also often used when creating documents that use multiple languages (such as this book). However, in-memory interoperability has some drawbacks as it requires the analyst to be familiar with setting up and using both environments, more complex objects are often not supported by both languages and there is a greater memory overhead as data can easily become duplicated (making it difficult to use for larger datasets).

### Interoperability between R ecosystems

Before we look at in-memory interoperability between R and Python first let’s consider the simpler case of converting between the two R ecosystems. The **{Seurat}** package provides functions for performing this conversion [as described in this vignette](https://satijalab.org/seurat/articles/conversion_vignette.html).

In [6]:
%%R
sce_from_seurat <- Seurat::as.SingleCellExperiment(seurat)
sce_from_seurat


class: SingleCellExperiment 
dim: 2000 100 
metadata(0):
assays(2): X logcounts
rownames(2000): Gene-0 Gene-1 ... Gene-1998 Gene-1999
rowData names(0):
colnames(100): Cell_0 Cell_1 ... Cell_98 Cell_99
colData names(9): n_genes_by_counts log1p_n_genes_by_counts ...
  pct_counts_in_top_500_genes ident
reducedDimNames(2): PCA UMAP
mainExpName: NULL
altExpNames(0):


In [7]:
%%R
seurat_from_sce <- Seurat::as.Seurat(sce_from_seurat)
seurat_from_sce


An object of class Seurat 
2000 features across 100 samples within 1 assay 
Active assay: RNA (2000 features, 0 variable features)
 2 dimensional reductions calculated: PCA, UMAP


The difficult part here is due to the differences between the structures of the two objects. It is important to make sure the arguments are set correctly so that the conversion functions know which information to convert and where to place it.

In many cases it may not be necessary to convert a `Seurat` object to a `SingleCellExperiment`. This is because many of the core Bioconductor packages for single-cell analysis have been designed to also accept a matrix as input.

In [8]:
%%R
# Calculate Counts Per Million using the Bioconductor scuttle package
# with a matrix in a Seurat object
cpm <- scuttle::calculateCPM(Seurat::GetAssayData(seurat, slot = "counts"))
cpm[1:10, 1:10]


10 x 10 sparse Matrix of class "dgCMatrix"
                                                                       
 [1,] 602.1263 622.1264  600.5326 1416.439   .         .       965.8600
 [2,] 602.1263   .         .       613.435 618.2562  943.8910  609.1107
 [3,] 602.1263 982.8946 1202.6879  969.506 618.2562  943.8910 1219.1005
 [4,]   .        .       600.5326  613.435 976.3175  594.3451    .     
 [5,] 602.1263 622.1264    .      1221.384 618.2562  594.3451  609.1107
 [6,] 954.8394 982.8946 1202.6879  613.435 618.2562  943.8910 1219.1005
 [7,] 954.8394 622.1264  952.6379  613.435 618.2562    .       965.8600
 [8,]   .      982.8946    .         .     618.2562  594.3451    .     
 [9,] 954.8394   .       600.5326  613.435   .      1192.4227 1219.1005
[10,] 954.8394 622.1264  952.6379  613.435 618.2562  943.8910  609.1107
                                  
 [1,]  958.4081 599.5090  610.1969
 [2,]  958.4081 952.2526  610.1969
 [3,]    .        .       965.9120
 [4,]    .        .      

However, it is important to be sure you are accessing the right information and storing any results in the correct place if needed.

### Accessing R from Python

The Python interface to R is provided by the [**rpy2** package](https://rpy2.github.io/doc/latest/html/index.html). This allows you to access R functions and objects from Python. For example:

In [9]:
counts_mat = adata.layers["counts"].T
rpy2.robjects.globalenv["counts_mat"] = counts_mat
cpm = rpy2.robjects.r("scuttle::calculateCPM(counts_mat)")
cpm

<2000x100 sparse matrix of type '<class 'numpy.float64'>'
	with 126146 stored elements in Compressed Sparse Column format>

Common Python objects (lists, matrices, `DataFrame`s etc.) can also be passed to R.

If you are using a Jupyter notebook (as we are for this book) you can use the IPython magic interface to create cells with native R code (passing objects as required). For example, starting a cell with `%%R -i input -o output` says to take `input` as input, run R code and then return `output` as output.

In [10]:
%%R -i counts_mat -o magic_cpm
# R code running using IPython magic
magic_cpm <- scuttle::calculateCPM(counts_mat)


In [11]:
# Python code accessing the results
magic_cpm

<2000x100 sparse matrix of type '<class 'numpy.float64'>'
	with 126146 stored elements in Compressed Sparse Column format>

This is the approach you will most commonly see in later chapters. For more information about using **rpy2** please refer to [the documentation](https://rpy2.github.io/doc/latest/html/index.html).

To work with single-cell data in this way the [**anndata2ri** package](https://icb-anndata2ri.readthedocs-hosted.com/en/latest/) is especially useful. This is an extension to **rpy2** which allows R to see `AnnData` objects as `SingleCellExperiment` objects. This avoids unnecessary conversion and makes it easy to run R code on a Python object. It also enables the conversion of sparse **scipy** matrices like we saw above.

In this example, we pass an `AnnData` object in the Python session to R which views it as a `SingleCellExperiment` that can be used by R functions.

In [12]:
%%R -i adata
qc <- scuttle::perCellQCMetrics(adata)
head(qc)


/Users/luke.zappia/miniconda3/envs/interoperability2/lib/python3.9/functools.py:888: NotConvertedWarning: Conversion 'py2rpy' not defined for objects of type '<class 'NoneType'>'
  return dispatch(args[0].__class__)(*args, **kw)


        sum detected total
Cell_0 2005     1274  2005
Cell_1 1941     1233  1941
Cell_2 2011     1270  2011
Cell_3 1947     1268  1947
Cell_4 1933     1265  1933
Cell_5 2031     1289  2031


Note that you will still run into issues if an object (or part of it) cannot be interfaced correctly (for example if there is an unsupported data type). In that case, you may need to modify your object first before it can be accessed.

### Accessing Python from R

Accessing Python from an R session is similar to accessing R from Python but here the interface is provided by the [**{reticulate}** package](https://rstudio.github.io/reticulate/). Once it is loaded we can access Python functions and objects from R.

In [13]:
%%R
reticulate_list <- reticulate::r_to_py(LETTERS)
print(reticulate_list)
py_builtins <- reticulate::import_builtins()
py_builtins$zip(letters, LETTERS)


List (26 items)


If you are working in an [RMarkdown](https://rmarkdown.rstudio.com/) or [Quarto](https://quarto.org/) document you can also write native Python chunks using the **{reticulate}** Python engine. When we do this we can use the magic `r` and `py` variables to access objects in the other language (the following code is an example that is not run).

````
```{r}
# An R chunk that accesses a Python object
print(py$py_object)
```

```{python}
# A Python chunk that accesses an R object
print(r$r_object)
```
````

Unlike **anndata2ri**, there are no R packages that provide a direct interface for Python to view `SingleCellExperiment` or `Seurat` objects as `AnnData` objects.  However, we can still access most parts of an `AnnData` using **{reticulate}** (this code is not run).

```r
# Print an AnnData object in a Python environment
py$adata
```
```
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```
```r
# Alternatively use the Python anndata package to read a H5AD file
anndata <- reticulate::import("anndata")
anndata$read_h5ad(h5ad_file)
```
```
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```
```r
# Access the obs slot, pandas DataFrames are automatically converted to R data.frames
head(adata$obs)
```
```
       n_genes_by_counts log1p_n_genes_by_counts total_counts
Cell_0              1246                7.128496         1965
Cell_1              1262                7.141245         2006
Cell_2              1262                7.141245         1958
Cell_3              1240                7.123673         1960
Cell_4              1296                7.167809         2027
Cell_5              1231                7.116394         1898
       log1p_total_counts pct_counts_in_top_50_genes
Cell_0           7.583756                  10.025445
Cell_1           7.604396                   9.521436
Cell_2           7.580189                   9.959142
Cell_3           7.581210                   9.183673
Cell_4           7.614805                   9.718796
Cell_5           7.549083                  10.168599
       pct_counts_in_top_100_genes pct_counts_in_top_200_genes
Cell_0                    17.65903                    30.89059
Cell_1                    16.99900                    29.71087
Cell_2                    17.62002                    30.28601
Cell_3                    16.83673                    30.45918
Cell_4                    17.11889                    30.04440
Cell_5                    18.07165                    30.29505
       pct_counts_in_top_500_genes
Cell_0                    61.42494
Cell_1                    59.62114
Cell_2                    60.92952
Cell_3                    61.07143
Cell_4                    59.64480
Cell_5                    61.48577
```

As mentioned above the R **{anndata}** package provides an R interface for `AnnData` objects but it is not currently used by many analysis packages.

For more complex analysis that requires a whole object to work on it may be necessary to completely convert an object from R to Python (or the opposite). This is not memory efficient as it creates a duplicate of the data but it does provide access to a greater range of packages. The **{zellkonverter}** package provides a function for doing this conversion (note that, unlike the function for reading H5AD files, this uses the normal Python environment rather than a specially created one) (code not run). 


```r
# Convert an AnnData to a SingleCellExperiment
sce <- zellkonverter::AnnData2SCE(adata, verbose = TRUE)
sce
```
```
✔ uns$hvg$flavor converted [21ms]
✔ uns$hvg converted [62ms]
✔ uns$log1p converted [22ms]
✔ uns$neighbors converted [21ms]
✔ uns$pca$params$use_highly_variable converted [22ms]
✔ uns$pca$params$zero_center converted [31ms]
✔ uns$pca$params converted [118ms]
✔ uns$pca$variance converted [17ms]
✔ uns$pca$variance_ratio converted [17ms]
✔ uns$pca converted [224ms]
✔ uns$umap$params$a converted [15ms]
✔ uns$umap$params$b converted [17ms]
✔ uns$umap$params converted [80ms]
✔ uns$umap converted [115ms]
✔ uns converted [582ms]
✔ Converting uns to metadata ... done
✔ X matrix converted to assay [44ms]
✔ layers$counts converted [29ms]
✔ Converting layers to assays ... done
✔ var converted to rowData [37ms]
✔ obs converted to colData [23ms]
✔ varm$PCs converted [18ms]
✔ varm converted [49ms]
✔ Converting varm to rowData$varm ... done
✔ obsm$X_pca converted [17ms]
✔ obsm$X_umap converted [17ms]
✔ obsm converted [80ms]
✔ Converting obsm to reducedDims ... done
ℹ varp is empty and was skipped
✔ obsp$connectivities converted [21ms]
✔ obsp$distances converted [22ms]
✔ obsp converted [89ms]
✔ Converting obsp to colPairs ... done
✔ SingleCellExperiment constructed [241ms]
ℹ Skipping conversion of raw
✔ Converting AnnData to SingleCellExperiment ... done
class: SingleCellExperiment
dim: 2000 100
metadata(5): hvg log1p neighbors pca umap
assays(2): X counts
rownames(2000): Gene_0 Gene_1 ... Gene_1998 Gene_1999
rowData names(11): n_cells_by_counts mean_counts ... dispersions_norm
  varm
colnames(100): Cell_0 Cell_1 ... Cell_98 Cell_99
colData names(8): n_genes_by_counts log1p_n_genes_by_counts ...
  pct_counts_in_top_200_genes pct_counts_in_top_500_genes
reducedDimNames(2): X_pca X_umap
mainExpName: NULL
altExpNames(0):
```

The same can also be done in reverse:

```r
adata2 <- zellkonverter::SCE2AnnData(sce, verbose = TRUE)
adata2
```
```
ℹ Using the 'X' assay as the X matrix
✔ Selected X matrix [27ms]
✔ assays$X converted to X matrix [38ms]
✔ additional assays converted to layers [31ms]
✔ rowData$varm converted to varm [15ms]
✔ reducedDims converted to obsm [63ms]
✔ metadata converted to uns [23ms]
ℹ rowPairs is empty and was skipped
✔ Converting AnnData to SingleCellExperiment ... done
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'X_name', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```

## Interoperability for multimodal data

The complexity of multimodal data presents additional challenges for interoperability. Both the `SingleCellExperiment` (through "alternative experiments", which must match in the column dimension (cells)) and `Seurat` (using "assays") objects can store multiple modalities but the `AnnData` object is restricted to unimodal data.

To address this limitation, the `MuData` object (introduced in the [analysis frameworks and tools chapter]({ref}`analysis frameworks and tools chapter <introduction:analysis-frameworks>`) was developed as as an extension of `AnnData` for multimodal datasets. The developers have considered interoperability in their design. While the main platform for MuData is Python, the authors have provided the [MuDataSeurat R package](https://pmbio.github.io/MuDataSeurat/) for reading the on-disk H5MU format as `Seurat` objects and the [MuData R package](https://bioconductor.org/packages/MuData/) for doing the same with Bioconductor `MultiAssayExperiment` objects. This official support is very useful but there are still some inconsistencies due to differences between the objects. The MuData authors also provide a [Julia implementation](https://docs.juliahub.com/Muon/QfqCh/0.1.1/objects/) of `AnnData` and `MuData`.

Below is an example of reading and writing a small example `MuData` dataset using the Python and R packages.

### Python

In [14]:
# Read file
mdata = mudata.read_h5mu("../../datasets/original.h5mu")
print(mdata)

# Write new file
python_h5mu_file = Path(temp_dir.name) / "python.h5mu"
mdata.write_h5mu(python_h5mu_file)

MuData object with n_obs × n_vars = 411 × 56
  obs:	'louvain', 'leiden', 'leiden_wnn', 'celltype'
  var:	'gene_ids', 'feature_types', 'highly_variable'
  obsm:	'X_mofa', 'X_mofa_umap', 'X_umap', 'X_wnn_umap'
  varm:	'LFs'
  obsp:	'connectivities', 'distances', 'wnn_connectivities', 'wnn_distances'
  2 modalities
    prot:	411 x 29
      var:	'gene_ids', 'feature_types', 'highly_variable'
      uns:	'neighbors', 'pca', 'umap'
      obsm:	'X_pca', 'X_umap'
      varm:	'PCs'
      layers:	'counts'
      obsp:	'connectivities', 'distances'
    rna:	411 x 27
      obs:	'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'celltype'
      var:	'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
      uns:	'celltype_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
      obsm:	'X_pca', 'X

/Users/luke.zappia/miniconda3/envs/interoperability2/lib/python3.9/site-packages/anndata/_core/anndata.py:1230: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[key] = c


### R

#### Bioconductor

Read/write from/to a `MultiAssayExperiment` object

In [2]:
%%R
mae <- MuData::readH5MU("../../datasets/original.h5mu")
print(mae)

bioc_h5mu_file <- tempfile(fileext = ".h5mu")
MuData::writeH5MU(mae, bioc_h5mu_file)


A MultiAssayExperiment object of 2 listed
 experiments with user-defined names and respective classes.
 Containing an ExperimentList class object of length 2:
 [1] prot: SingleCellExperiment with 29 rows and 411 columns
 [2] rna: SingleCellExperiment with 27 rows and 411 columns
Functionality:
 experiments() - obtain the ExperimentList instance
 colData() - the primary/phenotype DataFrame
 sampleMap() - the sample coordination DataFrame
 `$`, `[`, `[[` - extract colData columns, subset, or experiment
 *Format() - convert into a long or wide DataFrame
 assays() - convert ExperimentList to a SimpleList of matrices
 exportClass() - save data to flat files


#### Seurat

Read/write from/to a `Seurat` object

In [3]:
%%R
seurat <- MuDataSeurat::ReadH5MU("../../datasets/original.h5mu")
print(seurat)

seurat_h5mu_file <- tempfile(fileext = ".h5mu")
MuDataSeurat::WriteH5MU(seurat, seurat_h5mu_file)


R[write to console]: The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)




    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Warnung:
R[write to console]:  Keys should be one or more alphanumeric characters followed by an underscore, setting key from prot to prot_

R[write to console]: Warnung:
R[write to console]:  No columnames present in cell embeddings, setting to 'MOFA_1:30'

R[write to console]: Warnung:
R[write to console]:  Keys should be one or more alphanumeric characters followed by an underscore, setting key from MOFA_UMAP_ to MOFAUMAP_

R[write to console]: Warnung:
R[write to console]:  All keys should be one or more alphanumeric characters followed by an underscore '_', setting key to MOFAUMAP_

R[write to console]: Warnung:
R[write to console]:  No columnames present in cell embeddings, setting to 'MOFAUMAP_1:2'

R[write to console]: Warnung:
R[write to console]:  No columnames present in cell embeddings, setting to 'UMAP_1:2'

R[write to console]: Warnung:
R[write to console]:  Keys should be one or more alphanumeric characters followed by an underscore, setting key from

An object of class Seurat 
56 features across 411 samples within 2 assays 
Active assay: prot (29 features, 29 variable features)
 1 other assay present: rna
 8 dimensional reductions calculated: MOFA, MOFA_UMAP, UMAP, WNN_UMAP, protPCA, protUMAP, rnaPCA, rnaUMAP


R[write to console]: Added .var['highly_variable'] with highly variable features

R[write to console]: Added .var['highly_variable'] with highly variable features



## Interoperability with other languages

Here we briefly list some resources and tools for the interoperability of single-cell data with languages other than R and Python.

### Julia

- [Muon.jl](https://docs.juliahub.com/Muon/QfqCh/0.1.1/objects/) provides Julia implementations of `AnnData` and `MuData` objects, as well as IO for the H5AD and H5MU formats
- [scVI.jl](https://github.com/maren-ha/scVI.jl) provides a Julia implementation of `AnnData` as well as IO for the H5AD format

### JavaScript

- [Vitessce](http://vitessce.io/) contains loaders from `AnnData` objects stored using the Zarr format
- The [kana family](https://github.com/jkanche/kana) supports reading H5AD files and `SingleCellExperiment` objects saved as RDS files

### Rust

- [anndata-rs](https://github.com/kaizhang/anndata-rs) provides a Rust implementation of AnnData as well as advanced IO support for the H5AD format


## Session information

## Python

In [4]:
import session_info

session_info.show()

## R

In [5]:
%%R
sessioninfo::session_info()


─ Session info ───────────────────────────────────────────────────────────────
 setting  value
 version  R version 4.2.3 (2023-03-15)
 os       macOS Ventura 13.4.1
 system   x86_64, darwin13.4.0
 ui       unknown
 language (EN)
 collate  C
 ctype    UTF-8
 tz       Europe/Berlin
 date     2023-11-15
 pandoc   2.19.2 @ /Users/luke.zappia/miniconda3/envs/interoperability2/bin/pandoc

─ Packages ───────────────────────────────────────────────────────────────────
 package              * version    date (UTC) lib source
 abind                  1.4-5      2016-07-21 [1] CRAN (R 4.2.3)
 Biobase                2.58.0     2022-11-01 [1] Bioconductor
 BiocGenerics           0.44.0     2022-11-01 [1] Bioconductor
 bit                    4.0.5      2022-11-15 [1] CRAN (R 4.2.3)
 bit64                  4.0.5      2020-08-30 [1] CRAN (R 4.2.3)
 bitops                 1.0-7      2021-04-24 [1] CRAN (R 4.2.3)
 cli                    3.6.1      2023-03-23 [1] CRAN (R 4.2.3)
 cluster                2.1

## References

```{bibliography}
:filter: docname in docnames
:labelprefix: int
```

## Contributors

We gratefully acknowledge the contributions of:

### Authors

* Luke Zappia

### Reviewers

* Lukas Heumos
* Isaac Virshup
* Anastasia Litinetskaya
* Ludwig Geistlinger
* Peter Hickey